In [28]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import lars_path, LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score, mean_squared_error
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
import requests, io, re

from patsy import dmatrices
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import partial_dependence, permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNet

In [29]:
data_oct = pd.read_csv('/Users/jacopobinati/Desktop/HM2/listings_OCT_FINAL.csv')
data_oct.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,...,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,ln_price,property_House,property_Private room,property_Shared Room
0,NaN,NaN,f,1.0,2.0,"['email', 'phone']",t,f,Bushwick,Brooklyn,...,5.00,1,0,1,0,0.06,4.174387,False,True,False
1,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,t,Hell's Kitchen,Manhattan,...,NaN,1,0,1,0,NaN,4.700480,False,True,False
2,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,f,Sunnyside,Queens,...,NaN,1,1,0,0,NaN,4.595120,True,False,False
3,NaN,NaN,f,1.0,2.0,"['email', 'phone', 'work_email']",t,t,Bedford-Stuyvesant,Brooklyn,...,5.00,1,1,0,0,0.03,4.248495,True,False,False
4,NaN,100%,t,1.0,1.0,"['email', 'phone']",t,t,Bedford-Stuyvesant,Brooklyn,...,4.92,1,1,0,0,1.26,5.135798,True,False,False


In [30]:
columns_to_drop = ['host_verifications', 'latitude', 'longitude', 'neighbourhood_group_cleansed', 'host_listings_count',
                   'host_total_listings_count', 'maximum_nights_avg_ntm', 'minimum_minimum_nights', 'calendar_last_scraped',
                   'first_review']

data_oct.drop(columns_to_drop, axis=1, inplace=True)

In [31]:
for binary in [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    "has_availability",
]:
    data_oct[binary] = data_oct[binary].map({"t": True, "f": False})

data_oct["f_property_type"] = data_oct["property_type"].astype("category")
data_oct["f_neighbourhood_cleansed"] = data_oct["neighbourhood_cleansed"].astype("category")
data_oct['n_bathroom'] = data_oct['bathrooms_text'].str.extract('(\d+\.?\d*)').astype(float)

In [32]:
numericals = [
    "accommodates",
    "review_scores_value",
    "number_of_reviews_ltm",
    "number_of_reviews_l30d",
    "review_scores_location",
    "review_scores_communication",
    "review_scores_checkin",
    "review_scores_cleanliness",
    "reviews_per_month",
    "minimum_nights",
    "beds"
]

for col in numericals:
    data_oct["n_" + col] = pd.to_numeric(data_oct[col], errors="coerce")

In [48]:
data_oct.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms_text,beds,...,d_Paid_parking_lot_off_premises,d_Private_hot_tub,d_Fireplace_guards,d_Resort_access,d_EV_charger,d_Changing_table,d_Beach_access,d_stationary_bike,d_Exercise_equipment_yoga_mat,d_Bay_view
0,NaN,NaN,False,True,False,Bushwick,Private room,1,1 shared bath,1.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,False,True,True,Hell's Kitchen,Private room,2,1 bath,1.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,False,True,False,Sunnyside,House,1,1 bath,1.0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,False,True,True,Bedford-Stuyvesant,House,2,1 bath,1.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,100%,True,True,True,Bedford-Stuyvesant,House,4,1 bath,2.0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
data_oct["n_review_scores_value"].fillna(data_oct["n_review_scores_value"].mean(), inplace=True)
data_oct["n_review_scores_location"].fillna(data_oct["n_review_scores_location"].mean(), inplace=True)
data_oct["n_review_scores_communication"].fillna(data_oct["n_review_scores_communication"].mean(), inplace=True)
data_oct["n_review_scores_checkin"].fillna(data_oct["n_review_scores_checkin"].mean(), inplace=True)
data_oct["n_reviews_per_month"].fillna(data_oct["n_reviews_per_month"].mean(), inplace=True)

data_apr = data_oct.assign(
    flag_review_scores_value=np.multiply(data_oct.n_review_scores_value.isna(), 1),
    n_review_scores_rating=data_oct.n_review_scores_value.fillna(
        np.mean(data_oct.n_review_scores_value.dropna())
    ),
    flag_review_scores_location=np.multiply(data_oct.n_review_scores_location.isna(), 1),
    n_review_scores_location=data_oct.n_review_scores_location.fillna(
        np.mean(data_oct.n_review_scores_location.dropna())
    ),

    flag_review_scores_communication=np.multiply(data_oct.n_review_scores_communication.isna(), 1),
    n_review_scores_communication=data_oct.n_review_scores_communication.fillna(
        np.mean(data_oct.n_review_scores_communication.dropna())
    ),

    flag_review_scores_checkin=np.multiply(data_oct.n_review_scores_checkin.isna(), 1),
    n_review_scores_checkin=data_oct.n_review_scores_checkin.fillna(
        np.mean(data_oct.n_review_scores_checkin.dropna())
    ),

    flag_reviews_per_month=np.multiply(data_oct.n_reviews_per_month.isna(), 1),
    n_reviews_per_month=data_oct.n_reviews_per_month.fillna(
        np.mean(data_oct.n_reviews_per_month.dropna())
    ),

    flag_review_scores_cleanliness=np.multiply(data_oct.n_review_scores_cleanliness.isna(), 1),
    n_review_scores_cleanliness=data_oct.n_review_scores_cleanliness.fillna(
        np.mean(data_oct.n_review_scores_cleanliness.dropna())
    ),
)

In [34]:
variable_names = data_oct.columns.tolist()
variable_table = pd.DataFrame(variable_names, columns=["Variable Names"])
pd.set_option('display.max_rows', None)
print(variable_table)

                                  Variable Names
0                             host_response_rate
1                           host_acceptance_rate
2                              host_is_superhost
3                           host_has_profile_pic
4                         host_identity_verified
5                         neighbourhood_cleansed
6                                  property_type
7                                   accommodates
8                                 bathrooms_text
9                                           beds
10                                     amenities
11                                         price
12                                minimum_nights
13                                maximum_nights
14                              has_availability
15                               availability_30
16                               availability_60
17                               availability_90
18                              availability_365
19                  

In [35]:
replace_str_dict = {
    '"' : "",
    ", " : ",",
    "\\\\" : "",
    ":" : "",
    "\\+" : "_",
}

data_oct["amenities"] = data_oct["amenities"].replace(replace_str_dict, regex=True).str.strip("[]").str.split(",")

In [36]:
def merge_items_with_keywords(amenities_list, merge_dict):
    merged_amenities = []
    for amenities in amenities_list:
        merged_item = []
        for amenity in amenities:
            lower_amenity = amenity.lower()
            for new_category, old_categories in merge_dict.items():
                if any(old_category in lower_amenity for old_category in old_categories):
                    merged_item.append(new_category)
                    break
            else:
                merged_item.append(amenity)
        merged_amenities.append(list(set(merged_item)))
    return merged_amenities

In [37]:

merge_dict = {
    'wifi': ['wifi'],
    'kitchen': ['kitchen', 'kitchenette'],
    'stove': ['stove'],
    'oven': ['oven'],
    'microwave': ['microwave'],
    'refrigerator': ['refrigerator', 'fridge'],
    'dishwasher': ['dishwasher'],
    'kettel': ['kettle'],
    'toaster': ['toaster'],
    'coffee': ['coffee maker', 'machine', 'coffee', 'espresso', 'nespresso'],
    'tv': ['tv'],
    'sound_system': ['speaker', 'sound'],
    'game_console': ['game console', 'ps2', 'ps3', 'ps4', 'ps5', 
                     'playstation', 'wii', 'xbox'],
    'baby': ['baby', 'toys'],
    'body_wash': ['body', 'soap', 'shower gel'],
    'shampoo': ['shampoo'],
    'conditioner': ['conditioner'],
    'hair dryer': ['hair dryer'],
    'laundry': ['washer', 'laundry'],
    'backyard': ['backyard'],
    'grill': ['grill'],
    'breakfast': ['breakfast'],
    'clothing_storage': ['clothing storage'],
    'ac': ['ac - split type ductless system', 'air conditioning', 'central air conditioning', 'window ac unit'],
    'heating': ['heating']
}
data_oct['amenities'] = merge_items_with_keywords(data_oct['amenities'], merge_dict)
dummies = data_oct['amenities'].str.join('|').str.get_dummies()
dummies.columns = "d_" + dummies.columns.str.replace('/', '_').str.replace(' ', '_').str.replace('-', '_').str.replace('\\\\', '')

In [38]:
dummies.head()

,d_*Also,d_000_BTU_in_the_Bedroom_and_12,d_100%_Vegan,d_2_5_years_old,d_24_hour_fitness_center,d_26.4_QT_25_L,d_5_10_years_old,d_9,d_AVON,d_Aesop,...,d_stationary_bike,d_stove,d_tesla_only,d_toaster,d_treadmill,d_tv,d_wardrobe,d_wifi,d_wood_burning,d_yoga_mat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0


In [39]:
top_150_columns = dummies.sum().sort_values(ascending=False).head(150).index
final_dummies = dummies[top_150_columns]
final_dummies.head()

,d_wifi,d_Smoke_alarm,d_kitchen,d_heating,d_ac,d_Essentials,d_tv,d_Carbon_monoxide_alarm,d_Hangers,d_Hot_water,...,d_Paid_parking_lot_off_premises,d_Private_hot_tub,d_Fireplace_guards,d_Resort_access,d_EV_charger,d_Changing_table,d_Beach_access,d_stationary_bike,d_Exercise_equipment_yoga_mat,d_Bay_view
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [40]:
data_oct = pd.concat([data_oct, final_dummies], axis=1)
data_apr.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,property_type,accommodates,bathrooms_text,beds,...,n_reviews_per_month,n_minimum_nights,n_beds,flag_review_scores_value,n_review_scores_rating,flag_review_scores_location,flag_review_scores_communication,flag_review_scores_checkin,flag_reviews_per_month,flag_review_scores_cleanliness
0,NaN,NaN,False,True,False,Bushwick,Private room,1,1 shared bath,1.0,...,0.060000,30,1.0,0,5.000000,0,0,0,0,0
1,NaN,NaN,False,True,True,Hell's Kitchen,Private room,2,1 bath,1.0,...,1.077735,30,1.0,0,4.621355,0,0,0,0,1
2,NaN,NaN,False,True,False,Sunnyside,House,1,1 bath,1.0,...,1.077735,30,1.0,0,4.621355,0,0,0,0,1
3,NaN,NaN,False,True,True,Bedford-Stuyvesant,House,2,1 bath,1.0,...,0.030000,45,1.0,0,5.000000,0,0,0,0,0
4,NaN,100%,True,True,True,Bedford-Stuyvesant,House,4,1 bath,2.0,...,1.260000,30,2.0,0,4.920000,0,0,0,0,0


In [41]:
basic_vars = [
    "n_accommodates",
    "n_beds",
    "n_bathroom",
    "n_beds",
    "f_property_type",
    "f_neighbourhood_cleansed"
]

# reviews
reviews = [
    "n_review_scores_value",
    "flag_review_scores_value",
    "n_review_scores_location",
    "flag_review_scores_location",
    "n_review_scores_communication",
    "flag_review_scores_communication",
    "n_review_scores_checkin",
    "flag_review_scores_checkin",
    "n_review_scores_cleanliness",
    "flag_review_scores_cleanliness",
    "n_reviews_per_month",
    "flag_reviews_per_month"
]

# Dummy variables
amenities = [col for col in data_apr if col.startswith("d_")]

# interactions for the LASSO
# from ch14
X1 = [
    "n_accommodates:f_property_type",
    "d_breakfast:f_property_type",
    "d_heating:n_accommodates",
    "d_ac:f_property_type",
]
# with neighbourhood
X2 = [
    "f_property_type:f_neighbourhood_cleansed",
    "n_accommodates:f_neighbourhood_cleansed",
    "d_wifi:f_neighbourhood_cleansed",
    "d_Smoke_alarm:f_neighbourhood_cleansed",
]

In [42]:
predictors_1 = basic_vars
predictors_2 = basic_vars + reviews + amenities
predictors_E = basic_vars + reviews + amenities + X1 + X2

In [43]:
data_oct.to_csv('/Users/jacopobinati/Desktop/HM2/listings_OCT_FINAL2.csv', index=False)

In [44]:
data_train, data_holdout = train_test_split(data_oct, train_size=0.8, random_state=42)

# OLS

In [45]:
print(data_oct[reviews].dtypes)

KeyError: "['flag_review_scores_value', 'flag_review_scores_location', 'flag_review_scores_communication', 'flag_review_scores_checkin', 'flag_review_scores_cleanliness', 'flag_reviews_per_month'] not in index"

In [46]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

ols_model = LinearRegression().fit(X,y)

#y_test, X_test = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

y_hat = ols_model.predict(X)

ols_rmse = mean_squared_error(y,y_hat,squared=False)
ols_rmse

PatsyError: Error evaluating factor: NameError: name 'flag_review_scores_location' is not defined
    price ~ n_accommodates + n_beds + n_bathroom + n_beds + f_property_type + f_neighbourhood_cleansed + n_review_scores_value + flag_review_scores_value + n_review_scores_location + flag_review_scores_location + n_review_scores_communication + flag_review_scores_communication + n_review_scores_checkin + flag_review_scores_checkin + n_review_scores_cleanliness + flag_review_scores_cleanliness + n_reviews_per_month + flag_reviews_per_month
                                                                                                                                                                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
mod1 = smf.ols("price ~ " + " + ".join(amenities), data=data_train)

In [ ]:
ols_model_coeffs_df = pd.DataFrame(
    ols_model.coef_.tolist()[0],
    index=X.design_info.column_names,
    columns=["ols_coefficient"],
).assign(ols_coefficient=lambda x: x.ols_coefficient.round(3))

In [ ]:
ols_model_coeffs_df

# LASSO

In [ ]:
lasso_model_cv = GridSearchCV(
    lasso_model,
    {"alpha": [i / 100 for i in range(1, 26, 1)]},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)
